In [9]:
# import meep as mp
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from numpy import cos, sin
import json

# create geometric and write it to a file

generate random numbers

In [2]:
np.random.seed(15)

mean= (0, 0, 0)
cov = [[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]]
num_crystal = 10
loc = np.random.multivariate_normal(mean, cov, (num_crystal))
theta_x = np.random.uniform(0, 2*np.pi, num_crystal)
theta_y = np.random.uniform(0, 2*np.pi, num_crystal)
theta_z = np.random.uniform(0, 2*np.pi, num_crystal)

In [3]:
R = np.empty((10, 3, 3))

Rx_matrix = np.empty((num_crystal, 3, 3))
Ry_matrix = np.empty((num_crystal, 3, 3))
Rz_matrix = np.empty((num_crystal, 3, 3))

for i in range(num_crystal):
    Rx_matrix[i, :, :] = np.array([[1, 0, 0],
                   [0, cos(theta_x[i]), -sin(theta_x[i])], 
                  [0, sin(theta_x[i]), cos(theta_x[i])]])
    
    Ry_matrix[i, :, :] = np.array([[cos(theta_y[i]), 0, sin(theta_y[i])], 
                  [0, 1, 0],
                  [-sin(theta_y[i]), 0, cos(theta_y[i])]])
    
    Rz_matrix[i, :, :] = np.array([[cos(theta_z[i]), -sin(theta_z[i]), 0],
                 [sin(theta_z[i]), cos(theta_z[i]), 0],
                 [0, 0, 1]])

    R[i, :, :] = np.matmul(np.matmul(Ry_matrix[i, :, :], Rx_matrix[i, :, :]), Rz_matrix[i, :, :])

# size_cell = mp.Vector3(2,2,2)
# size_solid = mp.Vector3(1, 1, 1)
# solid_region = mp.Block(size_solid, 
#                     center = mp.Vector3(0, 0, 0),
#                     material=mp.Medium(epsilon=8.5))
# size_crystal = mp.Vector3(0.2, 0.2, 0.2)

og_x = np.array([[1, 0, 0] for i in range(num_crystal)])
og_y = np.array([[0, 1, 0] for i in range(num_crystal)])
og_z = np.array([[0, 0, 1] for i in range(num_crystal)])

Rx_vector = np.empty((num_crystal, 3))
Ry_vector = np.empty((num_crystal, 3))
Rz_vector = np.empty((num_crystal, 3))

for i in range(num_crystal):
    Rx_vector[i, :] = np.matmul(R[i, :, :], og_x[i, :])
    Ry_vector[i, :] = np.matmul(R[i, :, :], og_y[i, :])
    Rz_vector[i, :] = np.matmul(R[i, :, :], og_z[i, :])

# geometry = [solid_region,]

# for i in range(num_crystal):
#     if (np.abs(loc[i, 0]) < size_solid[0] - size_crystal[0]/2 and 
#     np.abs(loc[i, 1]) < size_solid[1] - size_crystal[1]/2 and 
#     np.abs(loc[i, 2]) < size_solid[2] - size_crystal[2]/2):
#         geometry.append(mp.Block(
#             size_crystal,
#             center = mp.Vector3(loc[i, 0], loc[i, 1], loc[i, 2]),
#             e1 = Rx[i, :],
#             e2 = Ry[i, :],
#             e3 = Rz[i, :],
#             material=mp.Medium(epsilon=10.5)))



In [18]:
export_geo = True

if export_geo:
    file_name = 'geometry.peter'
    to_write = [loc, Rx_matrix, Ry_matrix, Rz_matrix, theta_x, theta_y, theta_z]
    for i in range(len(to_write)):
        to_write[i] = to_write[i].tolist()
    with open(file_name, 'w') as f:
        json.dump(to_write, f)
    
#\\ad.monash.edu\home\User045\dche145\Documents\Abaqus\geometry_shapes
#\\Client\D$\source\working_with_meep

Create the visualization for visual inspection

In [5]:
pml_layers = [mp.PML(0.3)]

source_pad = 0.25
source = [mp.Source(mp.ContinuousSource(wavelength=2*(11**0.5), width=20),
                   component=mp.Ez,
                   center=mp.Vector3(0.75, 0, 0),
                   size=mp.Vector3(0, 0.1, 0.1))]
sim = mp.Simulation(resolution=50,
                    cell_size=size_cell,
                    boundary_layers=pml_layers,
                    sources = source,
                    geometry=geometry)


Quick visulization of the generated geometry

In [7]:
sim.init_sim()
eps_data = sim.get_epsilon()

show_3d_geo = True
if show_3d_geo:
    from mayavi import mlab
    s = mlab.contour3d(eps_data, colormap="YlGnBu")
    mlab.show()

cheeky run of the geometry

In [7]:
sim.run(mp.at_beginning(mp.output_epsilon), 
       mp.to_appended("ez_small", mp.at_every(0.6, mp.output_efield_z)),
       until=20)

creating output file "./eps-000000.00.h5"...
creating output file "./ez_small.h5"...
Meep progress: 0.43/20.0 = 2.1% done in 4.0s, 182.1s to go
on time step 46 (time=0.46), 0.0869736 s/step
Meep progress: 1.2/20.0 = 6.0% done in 8.1s, 127.1s to go
on time step 121 (time=1.21), 0.0536495 s/step
Meep progress: 1.99/20.0 = 9.9% done in 12.1s, 109.9s to go
on time step 200 (time=2), 0.0509718 s/step
Meep progress: 2.82/20.0 = 14.1% done in 16.1s, 98.4s to go
on time step 283 (time=2.83), 0.0483065 s/step
Meep progress: 3.6/20.0 = 18.0% done in 20.4s, 93.1s to go
on time step 360 (time=3.6), 0.0549899 s/step
Meep progress: 4.3500000000000005/20.0 = 21.8% done in 24.5s, 88.1s to go
on time step 435 (time=4.35), 0.0539277 s/step
Meep progress: 5.1000000000000005/20.0 = 25.5% done in 28.5s, 83.3s to go
on time step 510 (time=5.1), 0.0537785 s/step
Meep progress: 5.86/20.0 = 29.3% done in 32.5s, 78.5s to go
on time step 586 (time=5.86), 0.0526459 s/step
Meep progress: 6.59/20.0 = 32.9% done in 

visulization of the output field

In [38]:
# ez_data = sim.get_array(center=mp.Vector3(), size=size_cell, component=mp.Ez)
# print(ez_data.shape)
get_gif_out = False

if get_gif_out:
    !h5ls ez.h5
    !mkdir temp
    !mv eps-000000.00.h5 temp/eps-000000.00.h5
    !mv ez.h5 temp/ez.h5
    import os
    os.chdir('temp')
    !h5topng -z 20 -t 0:332 -R -Zc dkbluered -a yarg -A eps-000000.00.h5 ez.h5
    !convert ez.t*.png ez.gif
    os.chdir('..')

(100, 100, 100)
ez                       Dataset {100, 100, 100, 333/Inf}
h5topng error: invalid colormap file
